In [4]:
import pandas as pd
import re
import sys
import os
import re

In [5]:
df = pd.read_excel("/Users/iyeeun/Documents/coreview_3/crawl/coupang/Coupang-reviews/가죽마우스패드 + 예쁜 노트북 가방 파우치 엘지그램 삼성 갤럭시북.xlsx")

In [7]:
df = df[df["리뷰 내용"] != "등록된 리뷰내용이 없습니다"]

In [12]:
from konlpy.tag import Okt

okt = Okt()

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.



CalledProcessError: Command '['/usr/libexec/java_home']' returned non-zero exit status 1.

In [8]:
from soynlp.word import WordExtractor


# WordExtractor를 사용하여 scores 생성
word_extractor = WordExtractor()
word_extractor.train(df['리뷰 내용'].tolist())
word_scores = word_extractor.extract()

# scores 사전 생성
scores = {word: score.cohesion_forward for word, score in word_scores.items()}

training was done. used memory 0.103 Gb 0.096 Gb
all cohesion probabilities was computed. # words = 684
all branching entropies was computed # words = 1469
all accessor variety was computed # words = 1469


In [9]:
from soynlp.tokenizer import LTokenizer
from soynlp.tokenizer import MaxScoreTokenizer

# LTokenizer 초기화
ltokenizer = LTokenizer(scores=scores)
maxscore_tokenizer = MaxScoreTokenizer(scores=scores)

In [10]:
stopwords_lst = ['등록된', '리뷰', '조금', '수', '것', '없습니다', '도움이', '정말', '살짝', '또', '너무', '잘', '같아요', '엄청',  '바로', '한', '계속', '구매', '넣고', '먹기', '있어서', '다', '넘', '저는', '그냥', '맛이', '좋아요', 
                '아주', '펜슬', '좀', '좋고', '진짜', '완전', '있는', '때', '프로타주', '이', '좋아하는', '더', '좋은', '있습니다', '입니다', 
                '같아요', '있어', '좋은', '같습니다', '좋네요', '입니다', '있어요', '괜찮', '아니', '그런',
                '같아', '좋습니다', '좋을', '있는데', '없어', '아니라', '좋은', '같은', '없는', '어요', '좋아', '좋앗', '입니', '있고', '없고',
                '좋았', '좋습니', '생각',  '있을', '있습니', '있었', '아닌', '같습니', '습니', '니다', '정도']

In [11]:
# sys.path.append(os.path.dirname('PyKoSpacing/'))

# spacing = Spacing()  # Spacing 객체 초기화
# pip install git+https://github.com/haven-jeon/PyKoSpacing.git


def preprocessing(df, review_column, istokenize=True, pos_tags=None, filter=None):
    """
    리뷰 데이터 전처리 함수.
    
    Args:
        df (pd.DataFrame): 입력 데이터프레임.
        review_column (str): 리뷰 내용이 있는 열의 이름.
        pos_tags (list[str]): 추출할 품사 태그 리스트 (예: ['Noun', 'Adjective']).
        istokenize (bool): True일 경우 토큰화 사용, False일 경우 원문을 그대로 처리.
        filter (str): 제거할 단어 리스트 (옵션).

    Returns:
        pd.DataFrame: 전처리된 결과를 추가한 데이터프레임.
    """
    #spaceing 객체 정의 및 초기화
    # spacing = Spacing()
    processed_reviews  = []
    #인풋리뷰
    for idx, r in enumerate(df[review_column]):
        #하나의 리뷰에서 문장 단위로 자르기
        #불필요한 정보제거 name의 각 단어를 review에서 제거 
        # 이름이 주어졌을 경우에만 이름 정보 제거
        sentence = r

        # #spacing 적용 
        # sentence = sentence.replace(" ", '')
        
        # sentence = spacing(sentence) 

        #특수문자, 영어 알파벳, 초성/중성/종성(예: "ㄱ", "ㅏ" 등)을 제거.
        sentence = re.sub('\n','',sentence)
        sentence = re.sub('\u200b','',sentence)
        sentence = re.sub('\xa0','',sentence)
        sentence = re.sub('([a-zA-Z])','',sentence)
        sentence = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',sentence)
        sentence = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',sentence)
        sentence = re.sub(r'\s+', ' ', sentence)  # 다중 공백 -> 단일 공백
        sentence = re.sub(r'[^\w\s]', '', sentence)  # 특수문자 제거
        sentence = re.sub(r'\d+', '', sentence)  # 숫자 제거
        #전처리 후 문장이 비어있으면 다음 리뷰로 넘어감 
        if len(sentence) == 0:
            continue
        
        # 토큰화 여부 확인
        if istokenize:
            tokens = ltokenizer.tokenize(sentence)  # LTokenizer를 이용한 토큰화
        else:
            tokens = [sentence]  # 토큰화하지 않고 원문 사용
        
        # 불용어 필터링
        if filter:
            tokens = [token for token in tokens if token not in filter]
        # 품사 필터링
        if pos_tags:
            filtered_tokens = []
            for token in tokens:
                pos = okt.pos(token, norm=True, stem=False)  # 품사 태깅
                filtered_tokens += [word for word, tag in pos if tag in pos_tags]

        else:
            filtered_tokens = []
            for token in tokens:
                pos = okt.pos(token, norm=True, stem=False)  # 품사 태깅
                filtered_tokens += [word for word, tag in pos]


        # 길이가 1인 단어 제거
        filtered_tokens = [token for token in filtered_tokens if len(token) > 1]

        # 토큰을 공백으로 연결하고 마침표 추가
        processed_sentence = ' '.join(filtered_tokens)
        processed_reviews.append(processed_sentence)
        
    return pd.Series(processed_reviews)  # 시리즈 형태로 반환